# Genetic Algorithm

Natural Selection based.

Initial Population -> Fitness Function -> Selection -> Crossover -> Mutation 

In [7]:
cities = [
    'F',  # "Fortaleza",
    'C',  # "Caucaia",
    'J',  # "Juazeiro do Norte",
    'M'  # "Maracanaú"
]
costs = [
    [0, 10, 100, 50],
    [10, 0, 20, 20],
    [100, 20, 0, 10],
    [50, 20, 10, 0]
]

# Idea: F -> C -> J -> M -> F

In [8]:
# Since we will always be back to initial state, doesn't seem smart to set start state inside the DNA  
dna = "123"  # == C J M

In [9]:
gene_example = "132"  # = CMJ
best_gene = "123"

In [11]:
def fitness(gene: str):
    cost = 0
    prev = 0
    for dna in gene:
        next = int(dna)
        cost += costs[prev][next]
        prev = dna